In [ ]:
pip install segmentation-models-pytorch

In [ ]:
pip install albumentations==0.5.2

In [ ]:
import os     # 文件操作API
import cv2    # opencvAPI
import pdb    # 程序调试API
import time   # 时间API
import warnings  # 警告API
import random    # 随机数生成API
import numpy as np
import pandas as pd
from tqdm import tqdm # 进度条
from sklearn.model_selection import train_test_split # 将数据集分成训练集和测试集的API
import torch
import torch.nn as nn                  #  神经网络包
from torch.nn import functional as F   #  神经网络的函数
import torch.optim as optim            #  优化器

#  设置 torch.backends.cudnn.benchmark=True 将会让程序在开始时花费一点额外时间，
#  为整个网络的每个卷积层搜索最适合它的卷积实现算法，进而实现网络的加速
import torch.backends.cudnn as cudnn
# 导入数据集相关的模块Dataset是导入外部数据集时使用的API，sampler用来对数据进行采样
from torch.utils.data import DataLoader, Dataset, sampler
# 画图API
from matplotlib import pyplot as plt
# albumentations 是数据增强的API
# HorizontalFlip 围绕Y轴水平翻转
# VerticalFlip 围绕X轴垂直翻转
# ShiftScaleRotate 随机应用仿射变换：平移，缩放和旋转
from albumentations import HorizontalFlip, VerticalFlip, ShiftScaleRotate
# 归一化，图片裁剪，组合器，高斯噪声
from albumentations import Normalize, Resize, Compose, GaussNoise
# 将图像数据从pillow转换到tensor模式
from albumentations.pytorch import ToTensor
# 图像分割的模型API
import segmentation_models_pytorch as smp
# 忽略警告
warnings.filterwarnings("ignore")



# 以下的操作都是保证实验结果具有可以复现性
seed = 69
# 将随机数种子锚定为69，设置随机数种子可以使每一次生成随机数据的时候结果相同
random.seed(seed)
# 主要是为了禁止hash随机化，使得实验可复现
os.environ["PYTHONHASHSEED"] = str(seed)
# 将np随机数种子锚定为69
np.random.seed(seed)
# 将GPU的随机数种子锚定为69
torch.cuda.manual_seed(seed)
# 每次返回的卷积算法是确定的，使cuda保证每次结果一样，使得实验可复现，也相当于锚定
torch.backends.cudnn.deterministic = True
# 定义图片的尺寸
SIZE = (256, 1600)

# 定义数据的路径
# 提交模板
sample_submission_path = '../input/severstal-steel-defect-detection/sample_submission.csv'
# 训练集标签
train_df_path = '../input/severstal-steel-defect-detection/train.csv'
# 原文件夹
data_folder = '../input/severstal-steel-defect-detection/'
# 训练集图片
train_data_folder = '../input/severstal-steel-defect-detection/train_images/'
# 测试集图片
test_data_folder = '../input/severstal-steel-defect-detection/test_images/'
# 读取csv文件成pandas.df的格式
train_df = pd.read_csv(train_df_path)

# RLE to Mask
# RLE转化成Mask
def rle2mask(rle, shape):
    # 获取出shape的宽和高
    height, width = shape[0], shape[1]
    # 创建出一个一维全0的np数组，数据个数是width*height，其中的数据类型是uint8型
    mask = np.zeros(width * height, dtype=np.uint8)

    # if rle is not np.nan:
    if not isinstance(rle, float):
        # 将str类型的rle中的字符串按照空格强行分开，返回一个字符串列表，用lable接受
        label = rle.split()

        # [m::n] #从a[m]开始，每跳 n 个取一个
        # map(int,字符串数组)是对字符串数组的每个字符串转化成int型，也就是整体生成一个int型的列表
        # 分别确定出游程编码的位置和长度，并且保存在int列表里
        positions = map(int, label[0::2])
        length = map(int, label[1::2])

        '''
        >>> a = [1,2,3]
        >>> b = [4,5,6]
        >>> c = [4,5,6,7,8]
        >>> zipped = zip(a,b)     # 打包为元组的列表
        [(1, 4), (2, 5), (3, 6)]
        '''

        # zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表
        for pos, le in zip(positions, length):
            # mask中对应的值变为1，由此实现了rle向mask的转换
            mask[pos:(pos + le)] = 1

    '''
    a = np.array([1,2,3,4,5,6])
    a
    array([1, 2, 3, 4, 5, 6])

    a.reshape(3,2)
    array([[1, 2],
       [3, 4],
       [5, 6]])

    a.reshape(3,2, order='F')
    array([[1, 4],
       [2, 5],
       [3, 6]])

    '''
    # 将一维数组转换成二维的，形成二维mask数组,order='F'是将数组进行竖着排列的
    # 为何要进行转置？因为rle的编码流程是：从上到下（越界则从左到右）
    return mask.reshape(shape[0], shape[1], order='F')


# Mask转化成RLE
def mask2rle(mask):
    # 将数组变成一维的(先进行.T转置，再进行压缩一维的处理)
    # 为何要进行转置？因为rle的编码流程是：从上到下（越界则从左到右）
    mask = mask.T.flatten()
    # 将数组进行复制
    pixels = np.concatenate([mask])

    """
    p = np.concatenate([[1,2,3,4,5]])
    p
    array([1, 2, 3, 4, 5])

    p[:-1]
    array([1, 2, 3, 4])

    p[1:]
    array([2, 3, 4, 5])

    runs = np.where(p[1:] != p[:-1])
    print(runs)
    type(runs)
    (array([0, 1, 2, 3, 4], dtype=int64),)
    tuple

    runs = np.where(p[1:] != p[:-1])[0]
    print(runs)
    type(runs)
    [0 1 2 3 4]
    numpy.ndarray
    """
    # np.where(条件)，若条件满足，返回索引值,但是返回的是一个元组(很坑)
    # 必须通过np.where(条件)[0]来将返回的值变成一个np数组类别
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    # 将runs中的值写入到字符串中，每个元素之间用空格隔开
    return ' '.join(str(x) for x in runs)


# Mask生成函数(参数分别为：竖直方向上的索引，csv文件的的df，以及图片的尺寸)
def make_mask(row_id, df, shape):
    # 将row_id一行的name给提取出来，保存fname中
    fname = df.iloc[row_id].name
    # 将row_id一行的所有标签给提取出来，保存labels中
    labels = df.iloc[row_id][:4]
    # 初始化定义对应高和宽的全0二维数组
    masks = np.zeros((shape[0], shape[1], 4), dtype=np.float32)

    for idx, label in enumerate(labels.values):
        # 给mask特征人工增加两个维度，因为在torch中的数据是(batch_size,通道数,宽,高)的格式
        masks[:, :, idx] = mask = rle2mask(label, shape)
    return fname, masks # 返回名称和masks


# 图片的处理器模块
def get_transforms(phase, mean, std):
    # 定义一个列表存放对数据进行处理的子处理器
    list_transforms = []
    # 就调用albumentations中的一些方法，并且传入一些参数，使之能对图片进行处理
    if phase == 'train':
        # 如果是训练集，就以0.5的比例进行水平和竖直翻转
        list_transforms.extend([HorizontalFlip(p=0.5), VerticalFlip(p=0.5)])
    # 同样是albumentations中的一些方法，对图片进行归一化(传入参数)和tensor化处理
    list_transforms.extend([Normalize(mean=mean, std=std, p=1),ToTensor()])
    # 将子处理器进行组合，形成一个大处理器，并且将大处理器当作该函数的返回值
    list_trfms = Compose(list_transforms)
    return list_trfms

# 将dataset类进行重写构成数据集生成器
class SteelDataset(Dataset):
    def __init__(self, df, data_folder, mean, std, phase):
        self.df = df  # 读取csv文件后生成的df
        self.root = data_folder  # 数据所在的文件夹路径
        self.mean = mean  # 平均值
        self.std = std    # 标准差
        self.phase = phase  # 训练集和测试集的标志
        self.transforms = get_transforms(phase, mean, std)  # 将均值和标准差传到图片处理器中
        self.fnames = self.df.index.tolist()  # 将df对象的索引变成一个列表来存放
        self.shape = SIZE  # 图片的尺寸

    # 定义该类的__getitem__函数(根据索引来返回处理后的数据)
    def __getitem__(self, idx):
        # 用两个参数来保存make_mask生成的名称和masks标签
        image_id, mask = make_mask(idx, self.df, self.shape)
        # 根据该图片的具体名称来定位图片路径
        image_path = os.path.join(self.root, 'train_images',  image_id)
        # 提取出该图片的BGR张量矩阵
        img = cv2.imread(image_path)
        # 用处理器对tensor图片和masks进行处理
        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']  # img来获取处理好的图片
        mask = augmented['mask']  # mask来获取处理好的mask
        # 将图片从BGR变成RGB格式
        mask = mask[0].permute(2, 0, 1)
        # 将处理好的数据进行返回
        return img, mask

    # 返回df对象的长度
    def __len__(self):
        return len(self.fnames)


# 定义数据加载器
def Data_Provider(data_folder, df_path, phase, mean=None, std=None, batch_size=4, num_workers=2):
    df = pd.read_csv(df_path)  # 根据路径创建csv文件的df对象
    # 将df对象ClassId列的数都变为int类型
    df['ClassId'] = df['ClassId'].astype(int)

    df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
    df['defects'] = df.count(axis=1)
    # 将数据集的标签分为训练标签和测试标签
    train, valid = train_test_split(df, test_size=0.2, stratify=df['defects'])
    # 如果标签是train就用train更新df,否则就用valid更新df
    df = train if phase == 'train' else valid
    # 生成数据集
    image_dataset = SteelDataset(df, data_folder, mean, std, phase)
    # 数据加载器
    dataloader = DataLoader(
        image_dataset,   # 数据集为刚刚生成的image_dataset
        batch_size=batch_size,
        num_workers=num_workers,  # 并行数
        pin_memory=True,          # 利用锁页内存，加快GPU的传输速度(如果PC的RAM过小就不能用了)
        shuffle=True)             # 打乱数据集
    # 将数据加载器返回
    return dataloader


# Dice分数计算函数(评估指标)
def dice_metric(probability, truth, threshold=0.5, reduction='none'):
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert (probability.shape == truth.shape)

        p = (probability > threshold).float().to(device)
        t = (truth > 0.5).float().to(device)
        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0).to(device)
        pos_index = torch.nonzero(t_sum >= 1).to(device)

        dice_neg = (p_sum == 0).float().to(device)
        dice_pos = 2 * (p * t).sum(-1) / ((p + t).sum(-1))
        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = torch.cat([dice_pos, dice_neg])
        dice_neg = np.nan_to_num(dice_neg.mean().item(), 0)
        dice_pos = np.nan_to_num(dice_pos.mean().item(), 0)
        dice = dice.mean().item()

        num_neg = len(neg_index)
        num_pos = len(pos_index)

    return dice, dice_neg, dice_pos, num_neg, num_pos


# IoU计算函数(评估指标)
def predict(X, threshold):  # 将实数输出结果转化为0-1格式的Mask
    Xp = np.copy(X)
    return (Xp > threshold).astype('uint8')


# 比较IoU
def compute_ious(pred, label, classes, ignore_index=255, only_present=True):
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c, label_c).sum()
        union = np.logical_or(pred_c, label_c).sum()
        if union != 0:
            ious.append(intersection / union)
    return ious if ious else [1]


def compute_iou_batch(outputs, labels, classes=None):
    ious = []
    preds = np.copy(outputs)
    labels = np.array(labels.cpu().numpy())
    for pred, label in zip(preds, labels):
        ious.append(np.nanmean(compute_ious(pred, label, classes)))
    iou = np.nanmean(ious)
    return iou

class Meter:
    def __init__(self, phase, epoch):
        self.base_threshold = 0.5
        self.base_dice_scores = []
        self.dice_neg_scores = []
        self.dice_pos_scores = []
        self.iou_scores = []

    # 更新函数，用于将iou得分存放在iou_scores列表中
    def update(self, targets, outputs):
        probs = torch.sigmoid(outputs)
        dice, dice_neg, dice_pos, _, _ = dice_metric(probs, targets, self.base_threshold)
        self.base_dice_scores.append(dice)
        self.dice_pos_scores.append(dice_pos)
        self.dice_neg_scores.append(dice_neg)
        preds = predict(probs, self.base_threshold)
        iou = compute_iou_batch(preds, targets, classes=[1])
        self.iou_scores.append(iou)

    def get_metrics(self):
        dice = np.mean(self.base_dice_scores)
        dice_neg = np.mean(self.dice_neg_scores)
        dice_pos = np.mean(self.dice_pos_scores)
        dices = [dice, dice_neg, dice_pos]
        iou = np.nanmean(self.iou_scores)
        return dices, iou

# 训练过程中每一epoch的日志记录，用于训练后的打印操作
def epoch_log(phase, epoch, epoch_loss, meter, start):
    dices, iou = meter.get_metrics()
    dice, dice_neg, dice_pos = dices
    print("Phase: %s | Epoch: %2d | Loss: %0.4f | IoU: %0.4f | dice: %0.4f"
          % (phase, epoch, epoch_loss, iou, dice))
    return dice, iou


class Trainer(object):
    '''This class takes care of training and validation of our model'''
    def __init__(self, model):
        self.num_workers = 6
        self.batch_size = {"train": 4, "valid": 2}     # 训练集和测试集的batch_size
        self.accumulation_steps = 128 // self.batch_size['train']  # gradient accumulation梯度累积
        self.lr = 1e-4  # 学习率
        self.num_epochs = 50  # 训练轮数
        self.best_loss = float("inf")  # 初始化最好的训练损失值(初始值是正无穷大)
        self.best_dice = 0.0
        self.phases = ["train", "valid"]  # 两个选项
        self.device = torch.device("cuda:0")
        # torch.set_default_tensor_type("torch.cuda.FloatTensor")
        self.net = model  # 神经网络模型
        self.criterion = torch.nn.BCEWithLogitsLoss()     # 损失函数
        # model.parameters()与model.state_dict()是查看网络参数的方法。一般来说，前者多见于优化器的初始化
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)   # 优化器
        self.max_lr_changes = 1
        self.lr_changes = 0
        self.valid_losses = []         # 测试集的loss列表
        self.patience = 2
        self.lr_reset_epoch = 0
        self.best_valid_loss = 1000.   # 初始化测试集上最好的loss
        self.net = self.net.to(self.device)   # 将神经网络模型转移到GPU上
        cudnn.benchmark = True   # 优化运行效率

        # 数据加载器(字典类型)  此处有两个数据加载器，一个对应数据集，一个对应测试集，两者也就batch_size不同
        self.dataloaders = {
            phase: Data_Provider(
                data_folder=data_folder,
                df_path=train_df_path,
                phase=phase,
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                batch_size=self.batch_size[phase],
                num_workers=self.num_workers)
            for phase in self.phases}
        self.losses = {phase: [] for phase in self.phases}      # 保存损失值的列表
        self.iou_scores = {phase: [] for phase in self.phases}  # 保存iou评分的列表
        self.dice_scores = {phase: [] for phase in self.phases} # 保存dice评分的列表

    #  返回函数:用于返回损失值和输出值
    def forward(self, images, targets):
        images = images.to(self.device)  # 将图片载入GPU
        masks = targets.to(self.device)  # 将目标mask载入GPU
        outputs = self.net(images)       # 神经网络模型的输出值
        loss = self.criterion(outputs, masks)   # 计算损失值
        # 将损失值和输出值进行返回
        return loss, outputs

    # 迭代函数:
    def iterate(self, epoch, phase):
        meter = Meter(phase, epoch)
        start = time.strftime("%H:%M:%S")    # 记录下开始的时间
        batch_size = self.batch_size[phase]  # 根据字典索引来取出对应的batch_size
        self.net.train(phase == "train")     # 如果选项为训练集就将网络切换到训练模式
        dataloader = self.dataloaders[phase] # 将构造方法中的数据载入器根据对应的字典索引取出来
        running_loss = 0.0                   # 初始化总loss的值
        total_batches = len(dataloader)      # 数据载入器中有多少个小batch
        # 如果标签是训练，就将数据加载器用进度条进行包装，使之在训练过程中内容可视化，否则就不包装
        tk = tqdm(dataloader, total=total_batches) if phase == "train" else dataloader
        # 清空网络中的梯度
        self.optimizer.zero_grad()

        # 该循环的作用就是用小的batch_size多次迭代来实现和大batch_size一样的效果(相当于训练一个epoch)
        # enumerate(tk) 将数据加上一个索引序列，每一个batch都有一个编号itr，其中有若干图像和目标的masks
        for itr, batch in enumerate(tk):
            # 在batch中取出图像和目标mask
            images, targets = batch
            # 将两者载入到GPU上
            images = images.to(device)
            targets = targets.to(device)
            # 将这些数据进行计算后再返回，得到损失值和输出值
            loss, outputs = self.forward(images, targets)
            # 损失函数与总计累积步数相除(为的是后面求加权,以及减小权重的改变量)
            loss = loss / self.accumulation_steps
            # print('\r itr: %s, loss: %s' % (itr, loss.detach().cpu().item()), end='')
            if phase == "train":
                # 如果标签是训练集的话就利用loss进行反向传播(但是因为Loss被整除了,所以网络中权重改变量要小不少)
                loss.backward()
                if (itr + 1) % self.accumulation_steps == 0:
                    # 如果达到了累积步数，就用优化器进行权重参数的更新，并且对梯度进行清零(相当于放大了batch_size)
                    self.optimizer.step()       # 对权重进行优化
                    self.optimizer.zero_grad()  # 清空网络中的梯度
                # 由于数据加载器被进度条进行了包装，set_description()的作用是再进度条的前面动态显示
                # 在显示的时候将loss进行了还原，item()返回的是tensor中的值，且只能返回单个值（标量），不能返回向量
                tk.set_description(f'train_loss (loss={loss.item() * self.accumulation_steps:.5f})')

            # 将每一个batch中的损失值(该损失值被大幅缩小)进行相加
            running_loss += loss.item()

            # detach()的作用是返回一个新的tensor，从当前计算图中分离下来的，但是仍指向原变量的存放位置
            # 而且得到的这个新tensor永远不需要计算其梯度，不具有grad
            # 但是使用detach返回的tensor和原始的tensor共同一个内存，即一个修改另一个也会跟着改变
            # cpu()的作用是将数据转移到CPU上
            outputs = outputs.detach().cpu()
            # 每计算完一个小batch就将iou_scores的得分存入列表中
            meter.update(targets, outputs)

        # 每一轮的小batch损失值的均值
        epoch_loss = (running_loss * self.accumulation_steps) / total_batches
        # 每一轮的两个评分
        dice, iou = epoch_log(phase, epoch, epoch_loss, meter, start)
        self.losses[phase].append(epoch_loss)
        self.dice_scores[phase].append(dice)
        self.iou_scores[phase].append(iou)
        torch.cuda.empty_cache()  # 对显存进行释放
        return epoch_loss


    # 训练开始的函数
    def start(self):
        # 进行多轮训练的循环
        for epoch in range(self.num_epochs):
            # 执行迭代，完成一个epoch，并返回该轮的平均loss
            train_loss = self.iterate(epoch, "train")
            # state是一轮训练完成后所保存的信息(在读取时也要按照格式来读取)
            state = {
                "epoch": epoch,                 # 训练轮数
                "best_loss": self.best_loss,    # 最佳损失值
                "state_dict": self.net.state_dict(),      # 模型权重的参数
                "optimizer": self.optimizer.state_dict()  # 优化器的一些参数
                }
            # 执行迭代，完成epoch，并返回测试集平均的loss
            valid_loss = self.iterate(epoch, "valid")
            # 返回测试集的dice评分
            valid_dice = self.dice_scores["valid"][epoch]
            # 将测试集的loss保存在valid_losses列表中
            self.valid_losses.append(valid_loss)
            if valid_loss < self.best_valid_loss:
                # 若有更小的测试集损失值，就更新最佳测试集损失值
                self.best_valid_loss = valid_loss

            # 深度学习的早停法
            elif (self.patience and epoch - self.lr_reset_epoch > self.patience and
                  min(self.valid_losses[-self.patience:]) > self.best_valid_loss):
                # "patience" epochs without improvement
                self.lr_changes += 1
                if self.lr_changes > self.max_lr_changes:  # 早期停止
                    break
                self.lr /= 5  # 学习率衰减
                self.lr_reset_epoch = epoch
                # 将优化器中的lr进行修改(若lr发生修改，对相应的优化器也要进行修改)以实现动态学习率
                self.optimizer.param_groups[0]['lr'] = self.lr
                print('lr updated to {}'.format(self.optimizer.param_groups[0]['lr']))
            if valid_dice > self.best_dice:
                # 若测试集上的dice评分大于最好的dice，就将state字典中新增best_dice索引
                # 将valid_dice的内容写入其中作为value，并且将best_dice进行更新，最后进行state的保存
                state["best_dice"] = self.best_dice = valid_dice
                torch.save(state, "./models/epoch{}-dice{:.4f}.pth".format(epoch, valid_dice))


if __name__ == '__main__':
    # 载入神经网络模型
    model = smp.FPN('efficientnet-b0', encoder_weights='imagenet', classes=4)
    if not os.path.exists('models'):
        os.makedirs('models')  # 用于储存模型
    # 定义计算场景(GPU)，并显示
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model = model.to(device)   # 将模型传入到GPU
    model_trainer = Trainer(model)   # 将模型传入到Trainer类中
    model_trainer.start()  # 开始训练